In [5]:
import osmnx as ox
import pandas as pd # Import pandas to help with potential float conversion issues

# --- Configuration ---
# Define search parameters
# Using coordinates for Times Square, New York City as an example
# Changing to Philadelphia for consistency with previous context
center_point = (39.9526, -75.1652)  # Latitude,Longitude
radius = 1500  # Search radius in meters
# OSM tag for bookstores is shop=books. We can also search for other related tags.
tags = {"shop": "books"}

# --- Search for POIs using osmnx ---
print(f"Searching for POIs with tags {tags} within {radius}m of {center_point}...")

try:
    # --- CORRECTED LINE FOR OSMNX >= 1.0 ---
    pois = ox.features.features_from_point(center_point, tags, dist=radius)

    # --- Process Results ---
    if pois.empty:
        print("No results found.")
    else:
        # Filter for points if the result contains polygons or lines
        pois_points = pois.copy() # Start with a copy to modify geometries

        # Convert non-point geometries to points (centroids for polygons, midpoints for lines)
        
        # For Polygons/MultiPolygons, use centroid
        poly_mask = pois_points.geom_type.isin(["Polygon", "MultiPolygon"])
        pois_points.loc[poly_mask, 'geometry'] = pois_points.loc[poly_mask, 'geometry'].centroid
        
        # For LineString/MultiLineString, use midpoint
        line_mask = pois_points.geom_type.isin(["LineString", "MultiLineString"])
        midpoints = []
        for g in pois_points.loc[line_mask, 'geometry']:
            if g.geom_type == "LineString":
                midpoints.append(g.interpolate(0.5, normalized=True))
            elif g.geom_type == "MultiLineString" and g.geoms:
                # Take the longest part of a MultiLineString and get its midpoint
                longest_part = max(g.geoms, key=lambda p: p.length)
                midpoints.append(longest_part.interpolate(0.5, normalized=True))
            else: # Fallback for empty or complex geometry
                midpoints.append(g.centroid)
        pois_points.loc[line_mask, 'geometry'] = midpoints
        
        # Now filter for actual points (after conversion)
        pois_points = pois_points[pois_points.geom_type == 'Point']

        if pois_points.empty:
            print("No actual point waypoints found after processing.")
        else:
            print(f"Successfully found {len(pois_points)} waypoints. Showing the first 5:")

            # Use .iloc to get specific row data
            for i in range(min(5, len(pois_points))):
                place = pois_points.iloc[i]
                name = place.get('name', 'N/A')
                # Get coordinates from the geometry object
                # For a Point geometry, .y is latitude, .x is longitude
                lat, lng = place.geometry.y, place.geometry.x

                # Try to construct an address from available tags
                # --- FIX: Explicitly convert parts to string ---
                addr_parts_raw = [
                    place.get('addr:housenumber'),
                    place.get('addr:street'),
                    place.get('addr:city'),
                    place.get('addr:postcode')
                ]
                
                # Filter out None values and explicitly convert remaining parts to string
                addr_parts = [str(part) for part in addr_parts_raw if pd.notna(part)] # Use pd.notna for robustness with numpy.nan/None

                # Join the parts
                address = ', '.join(addr_parts) if addr_parts else "Address not available"


                print(f"\n{i+1}. {name}")
                print(f"   Address: {address}")
                print(f"   Waypoint: ({lat}, {lng})")

except Exception as e:
    print(f"An error occurred: {e}")

Searching for POIs with tags {'shop': 'books'} within 1500m of (39.9526, -75.1652)...
Successfully found 9 waypoints. Showing the first 5:

1. Book Corner
   Address: 311, North 20th Street
   Waypoint: (39.9601132, -75.171529)

2. Books-A-Million
   Address: 901, Market Street
   Waypoint: (39.9513696, -75.1552838)

3. Joseph Fox Bookshop
   Address: 1724, Sansom Street, Philadelphia, 19103
   Waypoint: (39.9507535, -75.1697647)

4. Philly AIDS Thrift @ Giovanni's Room
   Address: 19107
   Waypoint: (39.9451729, -75.1611999)

5. Wooden Shoe Books
   Address: 704, South Street, Philadelphia, 19147
   Waypoint: (39.9421664, -75.1542044)


C:\Users\micha\AppData\Local\Temp\ipykernel_27668\1424729972.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois_points.loc[poly_mask, 'geometry'] = pois_points.loc[poly_mask, 'geometry'].centroid


In [ ]:
import osmnx as ox

# Helpful console logging
ox.settings.log_console = True

# Do NOT pass timeout to the function; set it globally
ox.settings.timeout = 180              # request timeout (seconds)
ox.settings.requests_per_minute = 120  # be polite; avoid rate limits

# If the default endpoint is flaky for you, switch to a mirror:
ox.settings.overpass_endpoint = "https://overpass.kumi.systems/api"  # reliable mirror

# Your tags
tags = {
    'shop': ['cafe','bakery','supermarket','general','convenience'],
    'amenity': ['restaurant','fast_food','cafe','pub','drinking_water'],
}

# Minimal point test (500 m around City Hall-ish coords)
point = (39.95315, -75.15915)
gdf_point = ox.geometries_from_point(point=point, tags=tags, dist=500)

# Full bbox query
north, south, east, west = 40.0125, 39.8938, -75.1084, -75.2099
gdf_bbox = ox.geometries_from_bbox(north, south, east, west, tags=tags)
print(len(gdf_point), len(gdf_bbox))


Searching for POIs with tags {'shop': ['cafe', 'bakery', 'supermarket', 'general', 'convenience'], 'amenity': ['restaurant', 'fast_food', 'cafe', 'pub', 'drinking_water']} within bounding box:
  North: 40.0125, South: 39.8938, East: -75.1084, West: -75.2099

Attempting to clear OSMnx cache...

--- TEST: Attempting a minimal query in a small area within the bbox ---
Querying for 'restaurant' near (39.95315, -75.15915000000001) within 500m...
ERROR during minimal point test: features_from_point() got an unexpected keyword argument 'timeout'
This suggests a fundamental problem with connecting to Overpass or the area.

--- MAIN QUERY: Attempting full bbox query with increased timeout ---
An error occurred during main bbox search: No matching features. Check query location, tags, and log.

Consider checking your internet connection or trying again later.
If the minimal point test also failed, it's likely a connectivity issue.


In [2]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, MultiLineString
import matplotlib.pyplot as plt # Make sure this is imported
import os # For potential manual cache clearing (though settings are better)

# --- Configuration for BBox Search ---
north = 40.0125
south = 39.8938
east = -75.1084
west = -75.2099

tags_bbox = {
    "shop": ["cafe", "bakery", "supermarket", "general", "convenience"],
    "amenity": ["restaurant", "fast_food", "cafe", "pub", "drinking_water"]
}

print(f"Searching for POIs with tags {tags_bbox} within bounding box:")
print(f"  North: {north}, South: {south}, East: {east}, West: {west}")

# --- Step 1: Configure OSMnx Settings (Corrected for newer versions) ---
print("\nAttempting to configure OSMnx settings (disable cache, enable logging, set timeout)...")
ox.settings.use_cache = False # Disable cache for this run
ox.settings.log_console = True # Enable console logging
ox.settings.timeout = 180 # Set a longer timeout for all queries

# You can also set a custom cache folder if needed:
# ox.settings.cache_folder = "./my_osmnx_cache" 
# To explicitly clear cache manually, you'd navigate to ox.settings.cache_folder and delete its contents.

# --- Step 2: Test with a minimal query within the bbox ---
print("\n--- TEST: Attempting a minimal query in a small area within the bbox ---")
test_lat = (north + south) / 2
test_lon = (east + west) / 2
test_tags = {"amenity": "restaurant"} # Very common tag

try:
    print(f"Querying for 'restaurant' near ({test_lat}, {test_lon}) within 500m...")
    # Timeout is now globally set via ox.settings.timeout, but can be overridden per function
    pois_test_point = ox.features.features_from_point((test_lat, test_lon), tags=test_tags, dist=500) 
    if not pois_test_point.empty:
        print(f"SUCCESS: Found {len(pois_test_point)} restaurants near center point as a test.")
        print(pois_test_point.head())
    else:
        print("FAIL: No restaurants found near center point, even in a small radius. This might indicate a broader issue.")
except Exception as e:
    print(f"ERROR during minimal point test: {e}")
    print("This suggests a fundamental problem with connecting to Overpass or the area.")

# --- Step 3: Main bbox query ---
print("\n--- MAIN QUERY: Attempting full bbox query ---")
try:
    pois_bbox_raw = ox.features.features_from_bbox(
        bbox=(north, south, east, west),
        tags=tags_bbox
    )

    if pois_bbox_raw.empty:
        print("No results found in the specified bounding box (after troubleshooting). This is unexpected.")
        print("\n--- VISUAL BBOX CHECK ---")
        print("Please visit a tool like https://boundingbox.klokantech.com/")
        print(f"And input the coordinates: {west}, {south}, {east}, {north} (min Long, min Lat, max Long, max Lat)")
        print("Confirm this box covers the area you expect in Philadelphia and has POIs.")
        print("You can also try using ox.geocode('Philadelphia, PA') to get coordinates for a known place.")

    else:
        # --- Process Results for BBox ---
        pois_bbox_processed = pois_bbox_raw.copy()

        # Convert non-point geometries to points
        poly_mask = pois_bbox_processed.geom_type.isin(["Polygon", "MultiPolygon"])
        pois_bbox_processed.loc[poly_mask, 'geometry'] = pois_bbox_processed.loc[poly_mask, 'geometry'].centroid
        
        line_mask = pois_bbox_processed.geom_type.isin(["LineString", "MultiLineString"])
        midpoints = []
        for g in pois_bbox_processed.loc[line_mask, 'geometry']:
            if g.geom_type == "LineString":
                midpoints.append(g.interpolate(0.5, normalized=True))
            elif g.geom_type == "MultiLineString" and g.geoms:
                longest_part = max(g.geoms, key=lambda p: p.length)
                midpoints.append(longest_part.interpolate(0.5, normalized=True))
            else:
                midpoints.append(g.centroid)
        pois_bbox_processed.loc[line_mask, 'geometry'] = midpoints
        
        pois_bbox_points = pois_bbox_processed[pois_bbox_processed.geom_type == 'Point']

        if pois_bbox_points.empty:
            print("No actual point waypoints found in bbox after processing (after troubleshooting).")
        else:
            print(f"Successfully found {len(pois_bbox_points)} waypoints in bbox. Showing the first 5:")

            for i in range(min(5, len(pois_bbox_points))):
                place = pois_bbox_points.iloc[i]
                name = place.get('name', 'N/A')
                lat, lng = place.geometry.y, place.geometry.x

                addr_parts_raw = [
                    place.get('addr:housenumber'),
                    place.get('addr:street'),
                    place.get('addr:city'),
                    place.get('addr:postcode')
                ]
                addr_parts = [str(part) for part in addr_parts_raw if pd.notna(part)]
                address = ', '.join(addr_parts) if addr_parts else "Address not available"

                print(f"\n{i+1}. {name}")
                print(f"   OSM ID: {place.name}")
                print(f"   OSM Type: {place.get('shop') or place.get('amenity') or 'N/A'}")
                print(f"   Address: {address}")
                print(f"   Waypoint: ({lat}, {lng})")

            print("\nGenerating a plot of found POIs...")
            # Note: graph_from_bbox will also respect the global timeout setting
            G = ox.graph.graph_from_bbox(north, south, east, west, network_type="drive")

            fig, ax = ox.plot_graph(G, show=False, close=True, bgcolor='lightgray',
                                    node_size=0, edge_color='gray', edge_linewidth=0.5,
                                    bbox=(north, south, east, west))
            
            ax.scatter(pois_bbox_points.geometry.x, pois_bbox_points.geometry.y,
                       c='red', s=50, zorder=3, label='POIs')
            
            ax.set_title("POIs within Bounding Box (Philadelphia)")
            ax.legend()
            plt.show()
            print("Plot generated.")

except Exception as e:
    print(f"An error occurred during main bbox search: {e}")
    print("\nConsider checking your internet connection or trying again later.")
    print("If the minimal point test also failed, it's likely a connectivity issue.")

Searching for POIs with tags {'shop': ['cafe', 'bakery', 'supermarket', 'general', 'convenience'], 'amenity': ['restaurant', 'fast_food', 'cafe', 'pub', 'drinking_water']} within bounding box:
  North: 40.0125, South: 39.8938, East: -75.1084, West: -75.2099

Attempting to configure OSMnx settings (disable cache, enable logging, set timeout)...

--- TEST: Attempting a minimal query in a small area within the bbox ---
Querying for 'restaurant' near (39.95315, -75.15915000000001) within 500m...
SUCCESS: Found 95 restaurants near center point as a test.
                                      geometry addr:housenumber  \
element id                                                        
node    1034412875  POINT (-75.16008 39.94893)             1123   
        1167105119  POINT (-75.16372 39.95073)             1388   
        1489697259   POINT (-75.1618 39.95008)              111   
        1702834404   POINT (-75.15417 39.9564)              238   
        1702834414  POINT (-75.15598 39.95

c:\Users\micha\AppData\Local\Programs\Python\Python313\Lib\site-packages\osmnx\_overpass.py:271: UserWarning: This area is 39,071 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


KeyboardInterrupt: 